In [1]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get her0 if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

Current working directory: /Users/lucada/Documents/qhack/qhack21


In [2]:
from utils.data_handling import load_data, get_combined_cov_pos

In [3]:
data = load_data()

from copy import copy
data_backup = copy(data)

Reading data... ok


In [4]:
event_id = 0

data = data[data.event_id == event_id]
covs = get_combined_cov_pos(data)
data

,event_id,time_to_tca,mission_id,risk,max_risk_estimate,max_risk_scaling,miss_distance,relative_speed,relative_position_r,relative_position_t,...,t_sigma_rdot,c_sigma_rdot,t_sigma_tdot,c_sigma_tdot,t_sigma_ndot,c_sigma_ndot,F10,F3M,SSN,AP
0,0,1.566798,5,-10.204955,-7.834756,8.602101,14923.0,13792.0,453.8,5976.6,...,0.147350,58.272095,0.004092,0.165044,0.002987,0.386462,89.0,83.0,42.0,11.0
1,0,1.207494,5,-10.355758,-7.848937,8.956374,14544.0,13792.0,474.3,5821.2,...,0.059672,57.966413,0.003753,0.164383,0.002933,0.386393,89.0,83.0,42.0,11.0
2,0,0.952193,5,-10.345631,-7.847406,8.932195,14475.0,13792.0,474.6,5796.2,...,0.039258,57.907599,0.003576,0.164352,0.002967,0.386381,89.0,83.0,42.0,11.0
3,0,0.579669,5,-10.337809,-7.845880,8.913444,14579.0,13792.0,472.7,5838.9,...,0.022066,57.993905,0.003298,0.164309,0.002918,0.386400,89.0,83.0,40.0,14.0
4,0,0.257806,5,-10.391260,-7.852942,9.036838,14510.0,13792.0,478.7,5811.1,...,0.015075,57.946717,0.003670,0.164172,0.003220,0.386388,89.0,83.0,40.0,14.0


In [22]:
from utils.linalg import so3_to_su2, to_so3
from encoding import encode 
import pennylane.numpy as np
import pennylane as qml

In [27]:
selected_ind = [0, 1]
wires = np.arange(5 * len(selected_ind))
dev = qml.device('default.qubit', wires=wires)

def layer(params):
    qml.templates.AngleEmbedding(params[:5], wires[:5], rotation='X')
    qml.templates.AngleEmbedding(params[5:10], wires[:5], rotation='Z')
    qml.templates.AngleEmbedding(params[10:15], wires[:5], rotation='X')
    # entangle position t0 with t1
    for ind in np.arange(3):
        q0 = wires[ind]
        q1 = wires[ind+5]
        qml.CNOT(wires=[q0, q1])
    # entangle cov(t0) with pos(t1)
    for ind in np.arange(3, 5):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CNOT(wires=[q0,q1])
    # entangle cov(t1) with pos(t1)
    for ind in np.arange(8, 10):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CNOT(wires=[q0,q1])
    
    qml.templates.AngleEmbedding(params[15:18], wires[5:8], rotation='X')
    qml.templates.AngleEmbedding(params[18:21], wires[5:8], rotation='Z')
    qml.templates.AngleEmbedding(params[21:24], wires[5:8], rotation='X')

@qml.qnode(dev)
def circ(covs, params):
    for ind, cov in enumerate(covs):
        encode(cov, wires=wires[5*ind:5*(ind+1)])
    layer(params)
    return [qml.expval(qml.PauliZ(wires=i)) for i in wires[5:8]]

def model(covs, params):
    out = circ(covs, params)
    alphas = 2*np.arccos(np.sqrt(.5 * (out[:3] + 1)))
    return alphas

circ(covs[:2], np.arange(24))
print(circ.draw())

 0: ──RX(0.408)───RX(0)──RZ(5)─────RX(10)───────────────────────╭C──────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 1: ──RX(0.505)───RX(1)──RZ(6)─────RX(11)───────────────────╭C──│───────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 2: ──RX(1.09)────RX(2)──RZ(7)─────RX(12)───────────────╭C──│───│───────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 3: ──H──────────╭C──────RZ(1.86)──RX(1.16)──RZ(-3.14)──│───│───│───RX(3)──RZ(8)──RX(13)──╭C──────╭C──────────╭C────────────────────────────────────────────────────────────┤     
 4: ─────────────╰X──────RX(4)─────RZ(9)─────RX(14)─────│───│───│─────────────────────────│───╭C──│───────╭C──│───────────╭C────────────────────────────────────────────────┤     
 5: ──RX(0.35)──────────────────────────────────────────│───│───╰X────────────────────────╰X──╰X──│───╭X─

In [5]:
from collections import defaultdict

data = data_backup
covs = get_combined_cov_pos(data)

TypeError: first argument must be callable or None

In [12]:
events = defaultdict(list)
for (_,row), cov in zip(data.iterrows(), covs):
    events[row.event_id].append(cov)

In [36]:
training_x = []
training_y = []
for event_id, cdms in events.items():
    if len(cdms) < 3:
        continue
    ind = 0
    while ind + 3 <= len(cdms):
        fst, snd, thd = cdms[ind:ind+3]
        training_x.append((fst, snd))
        eigs, _ = np.linalg.eigh(thd)
        eigs = np.log(eigs)/20
        training_y.append(eigs)
        ind += 3
len(training_x), len(training_y)

(50218, 50218)

In [43]:
np.random.seed(145315)
params = np.random.rand(30)

opt = qml.AdamOptimizer(0.3)

training_x = training_x[:100]
training_y = training_y[:100]

def cost(x, y, params):
    out = model(x, params)
    return np.linalg.norm(out - y)**2

def accuracy(train_x, train_y, params):
    mse = 0
    for x,y in zip(train_x, train_y):
        mse += np.sqrt(cost(x, y, params)) / np.linalg.norm(y)
    return mse

n_epoch = 100
for epoch in np.arange(n_epoch):
    def obj_fn(params):
        loss = 0
        for x, y in zip(training_x, training_y):
            loss += cost(x, y, params)
        return loss
    params, current_loss = opt.step_and_cost(obj_fn, params)
    mse = accuracy(training_x, training_y, params)

    print(f"Epoch {epoch}: {mse:.1%}")

Epoch 0: 20543.0%
Epoch 1: 20464.1%
Epoch 2: 20363.7%
Epoch 3: 20103.4%
Epoch 4: 19511.3%
Epoch 5: 18597.8%
Epoch 6: 17829.6%
Epoch 7: 17542.7%
Epoch 8: 17474.1%
Epoch 9: 17451.0%
Epoch 10: 17469.8%
Epoch 11: 17497.0%
Epoch 12: 17492.2%
Epoch 13: 17426.8%
Epoch 14: 17279.0%
Epoch 15: 17068.8%
Epoch 16: 16889.2%
Epoch 17: 16841.9%
Epoch 18: 16928.3%
Epoch 19: 17060.8%
Epoch 20: 17150.7%
Epoch 21: 17156.7%
Epoch 22: 17086.8%
Epoch 23: 16981.1%
Epoch 24: 16889.0%
Epoch 25: 16844.3%
Epoch 26: 16847.9%
Epoch 27: 16870.9%
Epoch 28: 16877.6%
Epoch 29: 16854.5%
Epoch 30: 16814.0%
Epoch 31: 16778.9%
Epoch 32: 16767.7%
Epoch 33: 16782.2%
Epoch 34: 16801.5%
Epoch 35: 16794.7%
Epoch 36: 16753.1%
Epoch 37: 16700.1%
Epoch 38: 16665.8%
Epoch 39: 16664.9%
Epoch 40: 16689.4%
Epoch 41: 16710.1%
Epoch 42: 16700.1%
Epoch 43: 16665.5%
Epoch 44: 16636.6%
Epoch 45: 16637.0%
Epoch 46: 16667.1%
Epoch 47: 16703.0%
Epoch 48: 16713.3%
Epoch 49: 16691.4%
Epoch 50: 16661.3%
Epoch 51: 16649.3%
Epoch 52: 16661.5%
Epo